### ML Engineering

In [ ]:
step 1: create a nat gateway
step 2: create a route table
step 3: direct traffic in the route table to the nat gateway
step 4: attach the route table to the vpc
step 5: attach the nat to the vpc
step 6: create

### Engineering

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
micro-batch 1, micro-batch 2,... micro-batch n

In [ ]:
step 1: parameter partrition
step 2: assign the partrition to ranks
step 3: device
step 4: local optimizer
step 5: sync

In [ ]:
class VocabParallelEmbedding(nn.Module):
    def __init__(self, num_embedding, embedding_dim):
        super().__init__()
        world_size = torch.distributed.get_world_size()
        
        self.num_embedding = num_embedding
        self.embedding_dim_per_partrition = embedding_dim // world_size
        
        self.weight = nn.Parameter(torch.empty(
            self.embedding_dim_per_partrition,
            self.num_embedding
        ))
        
        self.vocab_start_idx, self.vocab_end_idx = self.get_vocab_range(
            embedding_dim_per_partrition
        )
    
    def get_vocab_range(self, embedding_dim_per_partrition):
        rank = torch.distributed.get_rank()
        start_idx = rank*embedding_dim_per_partrition
        end_idx = start_idx+embedding_dim_per_partrition
        return start_idx, end_idx
    
    def forward(self, tokens):
        mask = (tokens < self.vocab_start_idx) | (tokens >= self.vocab_end_idx)
        tokens = tokens - self.vocab_start_idx
        tokens[mask] = 0.
        
        embeddings = F.embedding(tokens, weight=self.weight)
        mask_idxs = torch.where(mask == 0)[1]
        embeddings[mask_idxs] = 0.
        
        torch.distributed.all_reduce(embeddings)
        
        return embeddings

In [ ]:
class ColumnLinearParallel(nn.Module):
    def __init__(self, input_size, output_size, world_size):
        super().__init__()
        self.input_size = input_size
        self.output_size_per_partrition = output_size // world_size
        
        self.weight = nn.Parameter(torch.empty(
            self.output_size_per_partrition,
            self.input_size
        ))
        self.bias = nn.Parameter(torch.empty(
            self.output_size_per_partrition
        ))
    
    def forward(self, input):
        output_parallel = F.linear(input, self.weight, self.bias)
        world_size = torch.distributed.get_world_size()
        outputs = [torch.empty_like(output_parallel) for _ in range(world_size)]
        torch.distributed.all_gather(outputs, output_parallel)
        outputs = torch.cat(outputs, dim=-1)
        return outputs

In [ ]:
def gather_tensor(x):
    world_size = torch.distributed.get_world_size()
    xs = [torch.empty_like(x) for _ in range(world_size)]
    torch.distributed.all_gather(xs, x)
    xs = torch.cat(xs, dim=-1)
    return xs

In [ ]:
step 1: determine the location of checkpoints
step 2: forward, save the checkpoint
step 3: delete
step 4: compute the grad
step 5: recompute the forward from the clostest checkpoint
step 6: 

In [ ]:
forward computing, recompute, gradientb

In [ ]:
class Model(nn.Module):
    def forward(self, x, labels):
        outputs = self.net(x)
        losses = loss_func(outputs, labels)
        return outputs, losses

In [ ]:
_, losses = model(x, labels)
loss = losses.mean()
loss.backward()

In [ ]:
os boot
user
process spawn

In [ ]:
register > cache (sram) > main memory (dram) >

In [ ]:
from torch.multiprocessing import Process

In [ ]:
def target(rank, world_size, config):
    torch.distributed.init_process_group(
        **config,
        rank=rank,
        world_size=world_size
    )

In [ ]:
world_size = 4

for rank in range(world_size):
    process = Process(target=target, args=(rank, world_size, config))

In [ ]:
from torch.multiprocessing import Process

In [ ]:
for _ in range(world_size):
    process = Process(target=say_hello)
    process.start()

In [ ]:
process_to_gpu = []

In [ ]:
for rank in range(world_size):
    device = rank % num_gpus
    process_to_gpu.append(device )

In [ ]:
step 1: partrition
step 2: move
step 3: init local
step 4: sync lococal and global
step 5: move local params to a bucket

In [ ]:
for rank in range(world_size):
    p = Process(target=say_hello, args=(rank,))
    p.start()

In [ ]:
tensor_model_parallel_size = 2
num_tensor_model_parallel_groups = 8

In [ ]:
list_ranks = []

In [ ]:
for i in range(num_tensor_model_parallel_groups):
    ranks = list(range(
        i*tensor_model_parallel_size,
        (i+1)*tensor_model_parallel_size
    ))
    print(ranks)

[0, 1]
[2, 3]
[4, 5]
[6, 7]
[8, 9]
[10, 11]
[12, 13]
[14, 15]


In [ ]:
world_size = 16
tensor_model_parallel_size = 2
pipeline_model_parallel_size = 4

In [ ]:
num_pipeline_model_parallel_groups = world_size // pipeline_model_parallel_size

In [ ]:
num_pipeline_model_parallel_groups

4

In [ ]:
data_parallel_groups = []

In [ ]:
for i in range(num_pipeline_model_parallel_groups):
    start_rank = i*

In [ ]:
register > cache sram > main memory dram > hard disk > backup

In [ ]:
- clock cycle 1: F_{1, 1}
- clock cycle 2: F_{1, 2}, F_{2, 1}
- clock cycle 3: F_{1, 3}, F_{2, 2}, F_{3, 1}

In [ ]:
new > ready > running > blocked > finished

In [ ]:
different consecutive stages
in the same replicas model 

In [ ]:
parallel_group
world_size

In [ ]:
tensor model parallel, pipeline model parallel, data parallel

In [ ]:
4 process
init
comminicate through the ___

In [ ]:
for i in range(pipeline_model_parallel_size):
    start_rank = i*num_pipeline_model_parallel_groups
    end_rank = (i+1)*num_pipeline_model_parallel_groups
    
    for j in range(tensor_model_parallel_size):
        ranks = list(range(
            start_rank+j,
            end_rank,
            tensor_model_parallel_size
        ))
        
        print(ranks)

[0, 2]
[1, 3]
[4, 6]
[5, 7]
[8, 10]
[9, 11]
[12, 14]
[13, 15]


In [ ]:
same portion
different replicas

In [ ]:
step 1: local env
step 2: init
step3: cuda
setp 4:

In [ ]:
rank, world_size, master_addr, master_port

### ML Engineering

In [ ]:
step 1: attach internet gateway to vpc
step 2: create a route table
step 3: direct traffic to the internet gateway
step 4: create a subnet
step 5: attach the route table to the subnet

In [ ]:
nat
subvpc
route
traffic
attach route to subvpc

In [ ]:
download
in-memory using apache arrow
extract schema
modify schema if needed
create a data catalog from the schema using aws glue

In [ ]:
from metaflow import FlowSpec, step, project

In [ ]:
@project(name="project_69")
class TrainFlow(FlowSpec):
    @step
    def start(self):
        self.next(self.train)
    
    @step
    def train(self):
        train()
        self.next(self.end)
    
    @step
    def end(self):
        pass

In [ ]:
import subprocess

In [ ]:
subprocess.run(["echo", "hello world"])

hello world


CompletedProcess(args=['echo', 'hello world'], returncode=0)

In [ ]:
from functools import reduce

In [ ]:
result = reduce(lambda x, y: x+y, numbers)

In [ ]:
import pytest

In [ ]:
@pytest.mark.parametrize(
    "input, output",
    [(2, 1), (2, 4)]
)

In [ ]:
from typing import cast, List

In [ ]:
numbers = cast(List[int], numbers)

In [ ]:
def outer_func():
    x = "hello"
    def inner_func():
        nonlocal x
        x = "shivon"
    
    inner_func()
    print(x)

In [ ]:
outer_func()

shivon


In [ ]:
step 1: num
step 2: 

In [ ]:
step 1: create a network
step 2: attach the network
step 3: receive the ips
step 4: communicate through the ips

In [ ]:
value, key, timestamp

### Engineering

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
class ParallelMLP(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.dense_h_to_4h = ColumnParallelLinear(
            input_size=hidden_size,
            ouput_size=hidden_size*4
        )
        self.gelu = nn.GELU()
        self.dense_4h_to_h = RowParallelLinear(
            input_size=hidden_size*4,
            output_size=hidden_size
        )
    
    def forward(self, input):
        intermediate_output = self.dense_h_to_4h(input)
        intermediate_output = self.gelu(intermediate_output)
        output = self.dense_4h_to_h(intermediate_output)
        return output

In [ ]:
in the same replicas model
different partrition of the same layer

In [ ]:
def is_grad_enabled(x):
    return torch.is_grad_enabled() and x.requires_grad

In [ ]:
def _broadcast(x):
    return x.clone()

In [ ]:
def _reduce(x):
    world_size = torch.distributed.get_world_size(group=parallel_group)
    torch.distributed.all_reduce(x)
    return x

In [ ]:
class Broadcast(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return _broadcast(input)
    
    @staticmethod
    def backward(ctx, grad_output):
        return _reduce(grad_output)

In [ ]:
def broadcast_with_forward_and_backward(inputs):
    if is_grad_enabled(inputs):
        outputs = Broadcast.apply(inputs)
    else:
        outputs = _broadcast(inputs)
    
    return outputs

In [ ]:
class VocabParallelEmbedding(nn.Module):
    def __init__(self, num_embedding, embedding_dim):
        super().__init__()
        world_size = torch.distributed.get_world_size()
        self.num_embedding = num_embedding
        self.embedding_dim_per_partrition = embedding_dim // world_size
        self.weight = nn.Parameter(torch.empty(
            self.embedding_dim_per_partrition
        ))
        
        self.vocab_start_

In [ ]:
step 1: load to disk
step 2: ids
step 3: memory size
step 4: reserve
step 5: load

In [ ]:
n_features = interference.shape[-1]

In [ ]:
polysemanticity[
    torch.arange(n_features),
    torch.arange(n_features)
] = 0.

In [ ]:
polysemanticity = polysemanticity.pow(2).sum(dim=-1).sqrt()

In [ ]:
tokens = model.to_tokens(text)

In [ ]:
seq_len = 5

In [ ]:
target_pattern = torch.zeros(seq_len, seq_len)

In [ ]:
target_pattern[torch.arange(seq_len), torch.arange(seq_len)-1] = 1.

In [ ]:
target_pattern

tensor([[0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]])

In [ ]:
consis

In [ ]:
repeated_tokens = model.to_tokens(repeated_text)

In [ ]:
_, cache = model(repeated_tokens)

In [ ]:
induction_heads = [(6, 9), (4, 2)]
attention_patterns = []

In [ ]:
from transformer_lens import utils

In [ ]:
for head_idx, layer_idx in induction_heads:
    act_name = utils.get_act_name("attn", layer_idx)
    layer_attention_pattern = cache[act_name]
    head_attention_pattern = layer_attention_pattern[:, head_idx]
    attention_patterns.append(head_attention_pattern)

In [ ]:
step 1: create two prompts
step 2: record activation
step 3: patch

In [ ]:
from einops import reduce

In [ ]:
induction_score = reduce(
    induction_stripe,
    "heads n_weights -> heads",
    reduction="mean"
)

In [ ]:
attn_weights.diagonal(
    dim1=-2,
    dim2=-1,
    offset=1-seq_len
)

In [ ]:
clean_tokens = model.to_tokens(clean_prompt)
corrupted_tokens = model.to_tokens(corrupted_prompt)

In [ ]:
correct_token = model.to_single_token(" John")
incorrect_token = model.to_single_token(" Mary")

In [ ]:
seq_len = clean_tokens.shape[-1]

In [ ]:
_, clean_activations = model.run_with_cache(clean_tokens)

In [ ]:
def patch_component(
    corrupted_activations,
    hook,
    position,
    clean_activations
):
    corrupted_activations[]

In [ ]:
from functools import partial

In [ ]:
from transformer_lens import utils

In [ ]:
def compute_logit_diff(logits, correct_token, incorrect_token):
    last_token_logits = logits[:, -1, :]
    correct_logit = last_token_logits[:, correct_token]
    incorrect_logit = last_token_logits[:, incorrect_token]
    return correct_logit - incorrect_logit

In [ ]:
patched_residual_stream_diff = torch.zeros(n_layers, seq_len)

In [ ]:
for layer_idx in range(n_layers):
    for position in range(seq_len):
        hook_name = utils.get_act_name("resid_pre", layer_idx)
        hook_func = partial(
            patch_component,
            position=position,
            clean_activations=clean_activations
        )
        
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(hook_name, hook_func)]
        )
        logit_diff = compute_logit_diff(patched_logits, correct_token, incorrect_token)
        patched_residual_stream_diff[layer_idx, position] = logit_diff

In [ ]:
step 1: create a clean prompt and corrupted prompt
step 2: record the activations of the clean prompt
step 3: run the model again with the corrupted prompt
step 4: patch
step 5: logit diff
step 6: identify

### AI

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
from transformers import DataCollatorForLanguageModeling

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer,
    mlm=False
)

In [ ]:
from functools import partial

In [ ]:
import jax

In [ ]:
@partial(jax.jit, static_argnames=["n"])
def demo():
    pass

In [ ]:
import ray

In [ ]:
@ray.remote
def fast_retrieve(x):
    return retrieve(x)

In [ ]:
data = [fast_retrieve.remote(x) for x in range(8)]

In [ ]:
from transformers import DataCollatorForLanguageModeling

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True
)

In [ ]:
from omegaconf import OmegaConf

In [ ]:
config = OmegaConf.load()

In [ ]:
from ray import tune
from ray.tune.schedulers import HyperBandScheduler

In [ ]:
scheduler = HyperBandScheduler()

In [ ]:
tune.run(
    objective,
    config=search_space,
    scheduler=scheduler
)

In [ ]:
target_state = torch.tensor([0, 0, 1])

In [ ]:
target_state.T 

In [ ]:
from jax import tree_map

In [ ]:
tree_map(square, xs)

In [ ]:
stream = torch.cuda.Stream(device=device)

In [ ]:
with torch.cuda.device(device):
    torch.cuda.stream(stream):
        mean = xs.mean(dim=-1)

In [ ]:
from gym import Env
from gym.spaces import Discrete, Box

In [ ]:
class ShowerEnv(Env):
    def __init__(self):
        super().__init__()
        self.action_space = Discrete(3)
        self.observation_space = Box

In [ ]:
conditioned, stimulus-evoke, motor, population

In [ ]:
logits = output.logits

In [ ]:
last_token_logits = logits[:, -1, :]

In [ ]:
last_token_logits[5]

In [ ]:
def compute_cross_entropy(logits, targets):
    log_probs = F.log_probs(logits, dim=-1)
    return -log_probs[torch.arange(targets), targets].mean()

In [ ]:
import gymnasium as gym

In [ ]:
gym.vector.SyncVectorEnv

In [ ]:
def probability_score(image_embedding, text_embedding):
    image_norm = image_embedding.norm(dim=-1)
    image_embedding = image_embedding / image_norm
    
    text_norm = text_embedding.norm(dim=-1)
    text_embedding = text_embedding / text_norm
    
    similarities = image_embedding @ text_embedding.T
    probs = F.probs(similarities, dim=-1)
    return probs

Step 1: Add the file to DVC.

Step 2: Update the `.gitignore` file to ignore the original file `x.txt`. Because we only want Git to track the DVC file (which is a small metadata file), not the original large data file.

Step 3: Commit the changes to Git.

Step 4: Push the dvc metadata file to remote storage

Step 5: Push the changes to Git.

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

In [ ]:
report = Report(metrics=[
    DataDriftPreset()
])

In [ ]:
report.run(reference=reference, current=current)

In [ ]:
step 1: add the file to dvc
step 2: add git
step 3: git ignore
step 4: dvc push
step 5: git push

In [ ]:
step 1: q&a prompt
step 2: yes and no
step 3: hidden states to probs
step 4: normalize
step 5: linear probe

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
def compute_cross_entropy(logits, targets):
    log_probs = F.log_probs(logits, dim=-1)
    return -log_probs[torch.arange(len(targets)), targets].mean()

In [ ]:
components = []

In [ ]:
text_embeddings = cache["hook_embed"]

In [ ]:
positional_embeddings = cache["hook_pos_embed"]

In [ ]:
components.append(text_embeddings)
components.append(positional_embeddings)

In [ ]:
for layer_idx in range(4):
    attn_name = get_act_name("attn_out", layer_idx)
    mlp_name = get_act_name("mlp_out", layer_idx)
    components.append(attn_name)
    components.append(mlp_name)

In [ ]:
discounted_returns = get_discounted_return(rewards)

In [ ]:
total_loss = 0.

In [ ]:
for discounted_return, prob in zip(discounted_return, selected_action_probs):
    total_loss += -discounted_return*prob.log()

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super().__init__()
        self.mha = MultiHeadAttention(d_model, n_heads)
        self.norm_1 = ResidualLayerNorm(d_model, dropout)
        self.feedforward = PositionWiseFeedForward(d_model, d_ff)
        self.norm_2 = ResidualLayerNorm(d_model, dropout)
    
    def forward(self, embeddings):
        attn_output, attn_pattern = self.mha(embeddings)
        norm_1 = self.norm_1(attn_output, embeddings)
        feedforward = self.feedforward(norm_1)
        norm_2 = self.norm_2(feedforward, norm_1)
        return norm_2

In [ ]:
class Optimizer:
    def __init__(self, params, lr):
        self.params = params
        self.lr = lr
    
    def step(self):
        for param in self.params:
            if param.requires_grad:
                pass

In [ ]:
5, 27, 27

In [ ]:
step 1: qa
step 2: yes no
step 3: hidden states
step 4: normalize
step 5: probe

In [ ]:
class Optimizer:
    def __init__(self, params, lr):
        self.params = params
        self.lr = lr
    
    def step(self):
        with torch.no_grad():
            for param in self.params:
                if param.requires_grad:
                    param -= param.grad * self.lr